In [1]:
from mmdet.apis import init_detector
import mmcv
from mmcv import Config


import copy
import os.path as osp

import numpy as np

from mmdet.datasets.builder import DATASETS
from mmdet.datasets.custom import CustomDataset

from mmdet.apis import set_random_seed


import json

/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import glob as _glob
import pandas as pd
import os
    
def glob(dir, pats, recursive=False):  # faster than match, python3 only
    pats = pats if isinstance(pats, (list, tuple)) else [pats]
    matches = []
    for pat in pats:
        matches += _glob.glob(os.path.join(dir, pat), recursive=recursive)
    return matches

In [3]:
#### load_annotations에서 뒤의 변수 받는거 custom dataset 에서는 이름을 바꿔도 되지만 아래에
#### configuration에서는 무조건 변수명을 ann_file로 받아야함
@DATASETS.register_module()
class Drive_dataset(CustomDataset):
    CLASSES=('car','bus','truck', 'special vehicle', 'motorcycle','bicycle','personal mobility','person','Traffic_light', 'Traffic_sign')


    def load_annotations(self, ann_file):
        
        CLASSES_dict = {'car' : 0 , 'bus' : 1, 'truck' : 2, 'special vehicle' : 3, 'motorcycle' : 4,'bicycle' : 5 ,'personal mobility' : 6 
                        ,'person' : 7 ,'Traffic_light' : 8, 'Traffic_sign' : 9}
        
        cat2label = {k: i for i, k in enumerate(self.CLASSES)}
        
        data_infos = []
        
        ls = pd.read_csv(ann_file, header = None)
        
        for idx,an in enumerate(ls.values):
            an=an[0]
            json_data = {}
            with open(an, "r") as json_file:
                json_data = json.load(json_file)
                
            ansplit = an.split('/')
            
            filename = ansplit[0] + '/' + ansplit[1] + '/' + 'images'+'/'+ json_data['image_name']
            
            width, height = json_data['image_size']

            data_info = dict(filename=filename, width=width, height=height)

            gt_bboxes = []
            gt_labels = []

            for ann_data in json_data['Annotation']:
                gt_labels.append(CLASSES_dict[ann_data['class_name']])
                gt_bboxes.append(ann_data['data'])


            data_anno = dict(
                    bboxes=np.array(gt_bboxes, dtype=np.float32).reshape(-1, 4),
                    labels=np.array(gt_labels, dtype=np.long))


            data_info.update(ann=data_anno)
            
            data_infos.append(data_info)
            
            if idx!=0 and idx%20000==0:
                print(str(idx)+'/'+str(len(ls))+' load annotations END!')
            
        
        
        return data_infos

In [4]:
## 추가수정 기존 받았던 pretrain과 매칭되는 config로 수정 
cfg = Config.fromfile('UniverseNet/configs/waymo_open/universenet50_2008_fp16_4x4_mstrain_640_1280_1x_waymo_open_f0.py') 

In [5]:
print(f'Config:\n{cfg.pretty_text}')

Config:
pretrained_ckpt = 'https://github.com/shinya7y/weights/releases/download/v1.0.2/res2net50_v1b_26w_4s-3cf99910_mmdetv2-92ed3313.pth'
model = dict(
    type='GFL',
    backbone=dict(
        type='Res2Net',
        depth=50,
        scales=4,
        base_width=26,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='SyncBN', requires_grad=True),
        norm_eval=False,
        style='pytorch',
        dcn=dict(type='DCN', deform_groups=1, fallback_on_stride=False),
        stage_with_dcn=(False, False, False, True),
        init_cfg=dict(
            type='Pretrained',
            checkpoint=
            'https://github.com/shinya7y/weights/releases/download/v1.0.2/res2net50_v1b_26w_4s-3cf99910_mmdetv2-92ed3313.pth'
        )),
    neck=[
        dict(
            type='FPN',
            in_channels=[256, 512, 1024, 2048],
            out_channels=256,
            start_level=1,
            add_extra_convs='on_output',
   

In [6]:
## 추가 및 수정 ## 
cfg.dataset_type  = 'Drive_dataset'
cfg.data_root = ''

## single GPU 이기 때문에 syncBN 이 아닌 BN으로 수정)
cfg.model.backbone.norm_cfg=dict(type='BN', requires_grad=True)

## Validation pipeline에 train pipeline 적용하기 위해서 구성 
cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(
        type='Resize',
        img_scale=(1920, 1200),
        multiscale_mode='range',
        keep_ratio=True),
    dict(type='RandomFlip', flip_ratio=0.0),
    dict(
        type='Normalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels'])
    
]

### test pipeline 나중에 test진행에 사용할 거 실제 validation은 위의 pipeline 으로 진행
cfg.test_pipeline = [
    ### TSET때 사용할 test time augmentation용 pipeline
    dict(type='LoadImageFromFile'),
    dict(
                type='MultiScaleFlipAug',
                img_scale=(1920, 1200),
                flip=False,
                transforms=[
                    dict(type='Resize', keep_ratio=True),
                    dict(type='RandomFlip'),
                    dict(
                        type='Normalize',
                        mean=[123.675, 116.28, 103.53],
                        std=[58.395, 57.12, 57.375],
                        to_rgb=True),
                    dict(type='Pad', size_divisor=32),
                    dict(type='ImageToTensor', keys=['img']),
                      dict(type='Collect', keys=['img'])
                ])
]

cfg.data=dict(
    samples_per_gpu=10,
    workers_per_gpu=12,
    train=dict(
        type=cfg.dataset_type,
        ann_file='2DBB/new_train.csv',
        pipeline=cfg.train_pipeline),
     val=dict(
        type=cfg.dataset_type,
        ann_file='2DBB/new_valid.csv',
        pipeline=cfg.test_pipeline),
    test=dict(
        type=cfg.dataset_type,
        ann_file='2DBB/new_valid.csv',
        pipeline=cfg.test_pipeline))

cfg.model.bbox_head.num_classes=10

cfg.device='cuda'
cfg.work_dir = 'checkpoints_ver2'

cfg.log_config.interval = 8000 #iteration 단위

cfg.seed = 2024

set_random_seed(cfg.seed, deterministic=False)

cfg.workflow = [('train', 1), ('val',1)]

cfg.evaluation = dict(interval=1, metric='mAP')

cfg.load_from = 'universenet50_2008_fp16_4x4_mstrain_480_960_2x_coco_20200815_epoch_24-81356447.pth'
cfg.runner = dict(type='EpochBasedRunner', max_epochs=24)

## NMS threshold 파라미터 수정 및 NMS type 수정
cfg.model.test_cfg['score_thr']=0.01
cfg.model.test_cfg.nms['iou_threshold']=0.3
cfg.model.test_cfg.nms['type']='soft_nms'
cfg.model.test_cfg.nms['min_score']=0.01
cfg.gpu_ids = range(1)

In [7]:
print(f'Config:\n{cfg.pretty_text}')

Config:
pretrained_ckpt = 'https://github.com/shinya7y/weights/releases/download/v1.0.2/res2net50_v1b_26w_4s-3cf99910_mmdetv2-92ed3313.pth'
model = dict(
    type='GFL',
    backbone=dict(
        type='Res2Net',
        depth=50,
        scales=4,
        base_width=26,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=False,
        style='pytorch',
        dcn=dict(type='DCN', deform_groups=1, fallback_on_stride=False),
        stage_with_dcn=(False, False, False, True),
        init_cfg=dict(
            type='Pretrained',
            checkpoint=
            'https://github.com/shinya7y/weights/releases/download/v1.0.2/res2net50_v1b_26w_4s-3cf99910_mmdetv2-92ed3313.pth'
        )),
    neck=[
        dict(
            type='FPN',
            in_channels=[256, 512, 1024, 2048],
            out_channels=256,
            start_level=1,
            add_extra_convs='on_output',
       

In [8]:
from mmdet.datasets import build_dataset, build_dataloader

In [9]:
test_dataloader_default_args = dict(
    samples_per_gpu=1, workers_per_gpu=2, dist=False, shuffle=False)

dataset = build_dataset(cfg.data.test)

test_loader_cfg = {
        **test_dataloader_default_args,
        **cfg.data.get('test_dataloader', {})
    }
# build the dataloader
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(dataset, **test_loader_cfg)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:43: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/root/UniverseNet/mmdet/datasets/custom.py:186: UserWarning: CustomDataset does not support filtering empty gt images.
  'CustomDataset does not support filtering empty gt images.')


In [10]:
from mmdet.models import build_detector
from mmcv.runner import (get_dist_info, init_dist, load_checkpoint,
                         wrap_fp16_model)

In [11]:
checkpoint_path = 'checkpoints_ver2/epoch_24.pth'

# build the model and load checkpoint
cfg.model.train_cfg = None
model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
fp16_cfg = cfg.get('fp16', None)
if fp16_cfg is not None:
    wrap_fp16_model(model)
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')
model.CLASSES = dataset.CLASSES

/root/UniverseNet/mmdet/models/dense_heads/anchor_head.py:116: UserWarning: DeprecationWarning: `num_anchors` is deprecated, for consistency or also use `num_base_priors` instead
  warnings.warn('DeprecationWarning: `num_anchors` is deprecated, '


load checkpoint from local path: checkpoints_ver2/epoch_24.pth


2024-08-16 06:40:43,708 - root - INFO - DeformConv2dPack backbone.layer4.0.convs.0 is upgraded to version 2.
2024-08-16 06:40:43,709 - root - INFO - DeformConv2dPack backbone.layer4.0.convs.1 is upgraded to version 2.
2024-08-16 06:40:43,710 - root - INFO - DeformConv2dPack backbone.layer4.0.convs.2 is upgraded to version 2.
2024-08-16 06:40:43,712 - root - INFO - DeformConv2dPack backbone.layer4.1.convs.0 is upgraded to version 2.
2024-08-16 06:40:43,713 - root - INFO - DeformConv2dPack backbone.layer4.1.convs.1 is upgraded to version 2.
2024-08-16 06:40:43,713 - root - INFO - DeformConv2dPack backbone.layer4.1.convs.2 is upgraded to version 2.
2024-08-16 06:40:43,715 - root - INFO - DeformConv2dPack backbone.layer4.2.convs.0 is upgraded to version 2.
2024-08-16 06:40:43,716 - root - INFO - DeformConv2dPack backbone.layer4.2.convs.1 is upgraded to version 2.
2024-08-16 06:40:43,717 - root - INFO - DeformConv2dPack backbone.layer4.2.convs.2 is upgraded to version 2.


In [12]:
from mmdet.utils import (build_ddp, build_dp, compat_cfg, get_device,
                         replace_cfg_vals, setup_multi_processes,
                         update_data_root)

from mmdet.apis import single_gpu_test ## 4070 super 단독 환경

In [13]:
model = build_dp(model, cfg.device, device_ids=cfg.gpu_ids)
outputs = single_gpu_test(model, data_loader)

[                               ] 3/10000, 3.9 task/s, elapsed: 1s, ETA:  2532s

/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448265233/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[>>>>>>>>>>>>>>>>>>>>>>>>] 10000/10000, 15.5 task/s, elapsed: 643s, ETA:     0s

In [14]:
# for oss in outputs:
#     for o in oss:
#         print(len(o))

In [15]:
mAP_output = dataset.evaluate(outputs,metric='mAP',iou_thr=0.5)


---------------iou_thr: 0.5---------------

+-------------------+-------+--------+--------+-------+
| class             | gts   | dets   | recall | ap    |
+-------------------+-------+--------+--------+-------+
| car               | 32445 | 356040 | 0.975  | 0.940 |
| bus               | 952   | 54775  | 0.966  | 0.901 |
| truck             | 5896  | 185796 | 0.969  | 0.890 |
| special vehicle   | 162   | 32602  | 0.772  | 0.510 |
| motorcycle        | 213   | 30697  | 0.864  | 0.751 |
| bicycle           | 81    | 10879  | 0.852  | 0.700 |
| personal mobility | 45    | 19153  | 0.889  | 0.806 |
| person            | 6890  | 115861 | 0.909  | 0.824 |
| Traffic_light     | 3932  | 58321  | 0.976  | 0.932 |
| Traffic_sign      | 2720  | 122145 | 0.958  | 0.862 |
+-------------------+-------+--------+--------+-------+
| mAP               |       |        |        | 0.812 |
+-------------------+-------+--------+--------+-------+


In [16]:
mAP_output

OrderedDict([('AP50', 0.743), ('mAP', 0.7431296110153198)])

In [21]:
!python UniverseNet/tools/analysis_tools/get_flops.py  UniverseNet/configs/waymo_open/universenet50_2008_fp16_4x4_mstrain_640_1280_1x_waymo_open_f0.py --shape  1920 1200

/root/UniverseNet/mmdet/models/dense_heads/anchor_head.py:116: UserWarning: DeprecationWarning: `num_anchors` is deprecated, for consistency or also use `num_base_priors` instead
  warnings.warn('DeprecationWarning: `num_anchors` is deprecated, '
/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448265233/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
GFL(
  24.288 M, 67.261% Params, 248.673 GFLOPs, 100.000% FLOPs, 
  (backbone): Res2Net(
    20.169 M, 55.856% Params, 202.432 GFLOPs, 81.405% FLOPs, 
    (stem): Sequential(
      0.0 M, 0.000% Params, 16.717 GFLOPs, 6.722% FLOPs, 
      (0): Conv2d(0.0 M, 0.000% Params, 0.504 GFLOPs, 0.203% FLOPs, 3, 32, ke